# Part 2 Analysis

In [ ]:
### NEED TO BE IN PYTHON 3.9 ENV FOR spacy TO WORK!!!! 

 #Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#import string
from nltk.stem.porter import PorterStemmer

In [ ]:
import numpy as np
import pandas as pd
general_log = pd.read_csv('reddit_post_comment_log.csv', index=False)

 initial count of keywords

In [ ]:
value_counts = general_log['keyword'].value_counts()
print(value_counts)

pie chart

In [ ]:
import matplotlib.pyplot as plt

# Data provided
keywords = ['rogue pouches', 'on! pouches', 'zyn', 'nicotine pouches', 'Lucy pouches', 
            'nic pouches', 'fre pouches', 'velo pouches', 'niin pouches', 'fr3sh pouches']
counts = [9094, 8253, 7304, 4871, 2963, 2180, 1642, 1391, 88, 11]
percentages = [count / sum(counts) * 100 for count in counts]
# Creating the pie chart
plt.figure(figsize=(8, 8))
plt.pie(counts, labels=keywords, autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)

# Customize font
plt.title("Distribution of Pouches by Keyword", fontsize=16, fontweight='bold', family='serif')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Set the font properties for the labels
plt.setp(plt.gca().texts, fontname='spectral', fontsize=11, fontweight='normal')

plt.show()

## text cleaning and lemmatization/tokenization

In [ ]:
general_log["post"] = general_log["post"].str.lower()
general_log["post"] = general_log["post"].replace(r'\?', '.', regex=True)
general_log.head()

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [ ]:
import nltk
import ssl

'''
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
'''
additional = ["nan", "else", "know", "take", "still", "seem", "mg", "go", "ago","maybe", "always", "put","thing","use","get","much", "www", "reddit", "moderator", "message", "https", "even", "lol", "com", "bot", "subreddit", "people", "delete", "sure", "really", "telegram", "askwoman", "comment","fuck","shit","literally","every_day","fucking","let","stuff","also", "com", "really", "https_www" ]
#nltk.download('stopwords')
stop_words = stopwords.words('english') + additional

In [ ]:
def remove_stopwords(texts):
    return [[word for word in doc if word not in stop_words] for doc in texts]

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV','PROPN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
def tokenize_lemmatization (text):
    data_words = list(sent_to_words(text))
    remove_stopwords_words = remove_stopwords(data_words)
    print("done_tokneize")
    bigram = gensim.models.Phrases(remove_stopwords_words, min_count=30, threshold=10) # higher threshold fewer phrases.
    #trigram = gensim.models.Phrases(bigram[data_words], threshold=50) 
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    #trigram_mod = gensim.models.phrases.Phraser(trigram)
    print("done_bigram")
    bigram_text = [bigram_mod[doc] for doc in remove_stopwords_words]
    
    
    data_lemmatized = lemmatization(bigram_text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV','PROPN'])
    # remove stopwords again
    remove_stopwords_words = remove_stopwords(data_lemmatized)
    
    return remove_stopwords_words

In [ ]:
general_log

In [ ]:
selected_text = general_log["post"]
clean_text = tokenize_lemmatization(selected_text)
clean_text 

## Sentiment Analysis

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    compound = sentiment_dict['compound']
    neg = sentiment_dict['neg']
    pos = sentiment_dict['pos']
    neu = sentiment_dict['neu']
    
    return compound, neg, pos,neu

 positive when score >0.05 , neutral when score <0.05 and >-0.05 , negative when <-0.05

In [ ]:
compounds = []
negs = []
pos_ = []
neus = []
i = 0
for i in range (len(selected_text)):
    if selected_text[i] is None or (isinstance(selected_text[i], float) and selected_text[i] != selected_text[i]):
        compounds.append(float('nan'))
        negs.append(float('nan'))
        pos_.append(float('nan'))
        neus.append(float('nan'))
    else:
        compound, neg, pos,neu  = sentiment_scores(selected_text[i])
        compounds.append(compound)
        negs.append(neg)
        pos_.append(pos)
        neus.append(neu)
        
general_log["compounds_score"] = compounds
general_log['pos_score'] = pos_
general_log['neu_score'] = neus
general_log['neg_score'] = negs
#categorize scores as neutral, positive, or negative
conditions = [
    (general_log['compounds_score'] <= -.05),
    (general_log['compounds_score'] > -.05) & (general_log['compounds_score'] < .05),
    (general_log['compounds_score'] >= .05)
]

values = ['negative', 'neutral', 'positive']

general_log['post_group'] = np.select(conditions, values)

general_log.head()

In [ ]:
general_log

In [ ]:
general_log.to_csv('reddit_combined_with_vader.csv', index=False)

In [ ]:
general_log = pd.read_csv('reddit_combined_with_vader.csv')

In [ ]:
#mean sentiment score
mean_sent = np.mean(general_log['compounds_score'])
print("Mean Sentiment Score is:", mean_sent)

#proportion neutral
prop_neut = len(general_log[general_log['post_group']=='neutral'])/len(general_log)
num_neut = len(general_log[general_log['post_group']=='neutral'])
print("Proportion neutral:", prop_neut)
print("Number neutral is:", num_neut )

#proportion positive
prop_pos =len(general_log[general_log['post_group']=='positive'])/len(general_log)
num_pos = len(general_log[general_log['post_group']=='positive']) 
print("Proportion positive:", prop_pos)
print("Number positive is:", num_pos)

#proportion negative
prop_neg =len(general_log[general_log['post_group']=='negative'])/len(general_log)
num_neg = len(general_log[general_log['post_group']=='negative'])
print("Proportion negative:", prop_neg)
print("Number negative is:",num_neg )
print(f"The sum of all posts n = {len(general_log)}")

### sentiment histogram 

proportion of three categories

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Data for plotting
categories = [f"Overall Posts (n = {len(general_log)})"]
scores_1 = [prop_pos]
scores_2 = [prop_neut]
scores_3 = [prop_neg]

# Width of each bar
bar_width = 0.2

# Positions of the bars on the x-axis
bar_positions_1 = np.arange(len(categories))
bar_positions_2 = bar_positions_1 + bar_width
bar_positions_3 = bar_positions_2 + bar_width

# Create the bar chart
plt.bar(bar_positions_1, scores_1, width=bar_width, color='orange', label='Prop Positive')
plt.bar(bar_positions_2, scores_2, width=bar_width, color='lightgreen', label='Prop Neutral')
plt.bar(bar_positions_3, scores_3, width=bar_width, color='lightblue', label='Prop Negative')

# Set x-axis tick labels and title
plt.xticks(bar_positions_2, categories)
plt.ylabel('Proportion')
plt.title('Proportion of Sentiment for All Posts.')

# Add a legend
plt.legend()

# Display the plot
plt.show()

proportion of overall compound scores

In [ ]:
# Compute mean
compound = general_log["compounds_score"]
mean_score = np.mean(compound)

# Plot histogram
plt.hist(compound, bins=20, edgecolor='black', alpha=0.7)

# Add mean as a vertical black dashed line
plt.axvline(mean_score, color='black', linestyle='dashed', linewidth=2, label=f'Mean: {mean_score:.2f}')

# Labels and title
plt.xlabel('VADER Sentiment Compound Score')
plt.ylabel('Frequency')
plt.title(f'Sentiment of Overall Posts (n = {len(general_log)})')

# Show legend
plt.legend()

# Show plot
plt.show()

revisting the distribution

## LDA

In [ ]:
clean_text

In [ ]:
# Create Dictionary
id2word_train = corpora.Dictionary(clean_text)
id2word_train.filter_extremes(no_below=4, no_above=0.95)

# Create Corpus
texts_a = clean_text

# Term Document Frequency
corpus = [id2word_train.doc2bow(text) for text in texts_a]
print(f"Dictionary size: {len(id2word_train)}")
print(f"Corpus size: {len(corpus)}")

In [ ]:
id2word_train.save("corpora")
id2word_train=corpora.Dictionary.load("corpora")

In [ ]:
import time
# coherence value
start_time = time.time()
coherence_values = []
model_list = []
for num_topics in range(2, 14):
    model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word_train,chunksize=50,
                                           random_state=100)
    print("--- %s seconds ---" % (time.time() - start_time))
    model_list.append(model)
    coherencemodel = CoherenceModel(model=model, texts=texts_a, dictionary=id2word_train, coherence='c_v')
    coherence_values.append(coherencemodel.get_coherence())
    print(num_topics)

In [ ]:
coherence_values

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model_list[1], corpus, id2word_train)
pyLDAvis.display(vis)

In [ ]:
topic_coords = vis.topic_coordinates
print(topic_coords)
# Create scatter plot
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(topic_coords['x'], topic_coords['y'], c='b', alpha=0.5)
ax.set_xlim([-0.4, 0.4])  # Adjust x-axis limits
ax.set_ylim([-0.4, 0.4])  # Adjust y-axis limits
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_title('Inter-Topic Distance Map')

# Add topic labels
for i, (x, y) in enumerate(zip(topic_coords['x'], topic_coords['y'])):
    ax.text(x, y, f'Topic {i}', fontsize=8, ha='center', va='center')

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Display the plot
plt.show()

In [ ]:
#Look at probabilities of top 20 words in each topic
model_list[1].print_topics(num_topics = 3, num_words = 20)

In [ ]:
# Get topic coordinates and sizes
topic_coords = vis.topic_coordinates
topic_sizes = vis.topic_info['Freq']  # Assuming 'Freq' represents the sizes

# Normalize topic sizes
topic_sizes_norm = (topic_sizes - topic_sizes.min()) / (topic_sizes.max() - topic_sizes.min())

# Create scatter plot with zoom and adjusted sizes
fig, ax = plt.subplots(figsize=(8, 6))
scatter = ax.scatter(topic_coords['x'], topic_coords['y'], c='b', alpha=0.5)  # Create scatter plot
ax.set_xlim([-0.6, 0.6])  # Adjust x-axis limits for zoom
ax.set_ylim([-0.6, 0.6])  # Adjust y-axis limits for zoom
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_title('Inter-Topic Distance Map for Overall Posts')

# Add topic labels
for i, (x, y) in enumerate(zip(topic_coords['x'], topic_coords['y'])):
    ax.text(x, y, f'Topic {i+1}', fontsize=8, ha='center', va='center')

# Adjust circle sizes
scatter.set_sizes(topic_sizes_norm * 10000)  # Adjust size scaling as needed

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Display the plot
plt.show()

In [ ]:
# create a word cloud for each topic where words that have higher probabilities are larger
from wordcloud import WordCloud


WordCloud = WordCloud(background_color="white",contour_width=3)
for t in range(model_list[1].num_topics):
    plt.figure()
    words =  model_list[1].show_topic(t, 100)
    words = dict((x,y) for x, y in words)
    plt.imshow(WordCloud.fit_words(words))
    plt.axis("off")
    plt.title("Post Topic #" + str(t+1))
    plt.show()

In [ ]:
topics = []
prob = []

for d in clean_text:
    bow = id2word_train.doc2bow(d)
    probs = model_list[1].get_document_topics(bow)
    max_prob = max([set[1] for set in probs])
    prob.append(max_prob)
    topics.append([set[0] for set in probs if set[1] == max_prob])

# Truncate topics list if needed
topics = topics[:len(general_log)]

# Assign to DataFrame
general_log["topics"] = [t[0] if len(t) > 0 else None for t in topics]
general_log["prob"] = prob

# Plot KDE for each topic
for i in range(3):
    general_log[general_log["topics"] == i]["prob"].plot(kind='kde', label=f"Topic {i+1}")

# Add legend, labels, title
plt.legend(title="Topics")
plt.xlabel("Probability")
plt.ylabel("Density")
plt.title("KDE of Probability by Topic")

# Show plot
plt.show()

In [ ]:
i = 0
while i < len(general_log):
    if general_log.at[i, "keyword"] == 'nicotine pouches' or general_log.at[i, "keyword"] == 'nic pouches' :
        post_text = general_log.at[i, 'post']
        if 'velo' in post_text:
            general_log.at[i, 'keyword'] = 'velo pouches'
        elif 'zyn' in post_text:
            general_log.at[i, 'keyword'] = 'zyn'
        elif 'fr3sh' in post_text:
            general_log.at[i, 'keyword'] = 'fr3sh pouches'
        elif 'niin' in post_text:
            general_log.at[i, 'keyword'] = 'niin pouches'
        elif 'on' in post_text:
            general_log.at[i, 'keyword'] = 'on! pouches'
        elif 'rogue' in post_text:
            general_log.at[i, 'keyword'] = 'rogue pouches'
        elif 'fre' in post_text:
            general_log.at[i, 'keyword'] = 'fre pouches'
    if general_log.at[i, "keyword"] == 'on pouches':
        general_log.at[i, 'keyword'] = 'on! pouches'
    if general_log.at[i, "keyword"] == 'nicotine pouches' or general_log.at[i, "keyword"] == 'nic pouches' :
        general_log.at[i, 'keyword'] = 'unspecified'
    i += 1


In [ ]:
general_log

In [ ]:
value_counts = general_log['keyword'].value_counts()
print(value_counts)

## sentiment by LDA

In [ ]:
#For easy understanding let's number the topics as 1,2,3 instead of 0,1,2
general_log['topic_final'] = general_log['topics']+1
print(general_log['topic_final'].value_counts())

#check that this worked
count_topic1 = pd.DataFrame(general_log['topic_final'].value_counts()).iloc[0][0]
count_topic2 = pd.DataFrame(general_log['topic_final'].value_counts()).iloc[2][0]
count_topic3 = pd.DataFrame(general_log['topic_final'].value_counts()).iloc[1][0]
#also check the propotion for each topic
print("Topic 1 count: n=", count_topic1, "prop = ", count_topic1/len(general_log))
print("Topic 2 count: n=", count_topic2, "prop = ", count_topic2/len(general_log))
print("Topic 3 count: n=", count_topic3, "prop = ", count_topic3/len(general_log))

In [ ]:
#get just topic 1
topic1 = general_log[general_log['topic_final']==1]

#mean sentiment score
mean_sent1 = np.mean(topic1['compounds_score'])
print("Mean Sentiment Score is:", mean_sent1)

#proportion neutral
num_neut1 = len(topic1[topic1['post_group']=='neutral'])
prop_neut1 = len(topic1[topic1['post_group']=='neutral'])/len(topic1)
print("Proportion neutral:", prop_neut1, "n:", num_neut1)

#proportion positive
num_pos1 =len(topic1[topic1['post_group']=='positive'])
prop_pos1 =len(topic1[topic1['post_group']=='positive'])/len(topic1)
print("Proportion positive:", prop_pos1, "n:", num_pos1)

#proportion negative
num_neg1 =len(topic1[topic1['post_group']=='negative'])
prop_neg1 =len(topic1[topic1['post_group']=='negative'])/len(topic1)
print("Proportion negative:", prop_neg1, "n:", num_neg1)

In [ ]:
#get just topic 2
topic2 = general_log[general_log['topic_final']==2]

#mean sentiment score
mean_sent2 = np.mean(topic2['compounds_score'])
print("Mean Sentiment Score is:", mean_sent2)

#proportion neutral
num_neut2 = len(topic2[topic2['post_group']=='neutral'])
prop_neut2 = len(topic2[topic2['post_group']=='neutral'])/len(topic2)
print("Proportion neutral:", prop_neut2, "n:", num_neut2)

#proportion positive
num_pos2 =len(topic2[topic2['post_group']=='positive'])
prop_pos2 =len(topic2[topic2['post_group']=='positive'])/len(topic2)
print("Proportion positive:", prop_pos2, "n:", num_pos2)

#proportion negative
num_neg2 =len(topic2[topic2['post_group']=='negative'])
prop_neg2 =len(topic2[topic2['post_group']=='negative'])/len(topic2)
print("Proportion negative:", prop_neg2, "n:", num_neg2)

In [ ]:
#get just topic 3
topic3 = general_log[general_log['topic_final']==3]

#mean sentiment score
mean_sent3 = np.mean(topic3['compounds_score'])
print("Mean Sentiment Score is:", mean_sent3)

#proportion neutral
num_neut3 = len(topic3[topic3['post_group']=='neutral'])
prop_neut3 = len(topic3[topic3['post_group']=='neutral'])/len(topic3)
print("Proportion neutral:", prop_neut3, "n:", num_neut3)

#proportion positive
num_pos3 =len(topic3[topic3['post_group']=='positive'])
prop_pos3 =len(topic3[topic3['post_group']=='positive'])/len(topic3)
print("Proportion positive:", prop_pos3, "n:", num_pos3)

#proportion negative
num_neg3 =len(topic3[topic3['post_group']=='negative'])
prop_neg3 =len(topic3[topic3['post_group']=='negative'])/len(topic3)
print("Proportion negative:", prop_neg3, "n:", num_neg3)

In [ ]:
general_log_topics_stat = pd.DataFrame(
    {'topic_number': [1, 2, 3],
        'mean_sent_score': [mean_sent1, mean_sent2, mean_sent3],
        'prop_neutral': [prop_neut1, prop_neut2, prop_neut3],
        'prop_positive': [prop_pos1, prop_pos2, prop_pos3],
        'prop_negative': [prop_neg1, prop_neg2, prop_neg3]}
)

general_log_topics_stat

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Data for three categories and their scores
categories = general_log_topics_stat['topic_number']
scores_1 = [prop_pos1, prop_pos2, prop_pos3]
scores_2 = [prop_neut1, prop_neut2,prop_neut3]
scores_3 = [prop_neg1, prop_neg2,prop_neg3]

# Width of each bar
bar_width = 0.2

# Positions of the bars on the x-axis
bar_positions_1 = np.arange(len(categories))
bar_positions_2 = bar_positions_1 + bar_width
bar_positions_3 = bar_positions_2 + bar_width

# Create the bar chart
plt.bar(bar_positions_1, scores_1, width=bar_width, color='orange', label='Prop Positive')
plt.bar(bar_positions_2, scores_2, width=bar_width, color='lightgreen', label='Prop Neutral')
plt.bar(bar_positions_3, scores_3, width=bar_width, color='lightblue', label='Prop Negative')

# Set x-axis tick labels and title
plt.xticks(bar_positions_2, categories)
plt.xlabel('Topics')
plt.ylabel('Proportion')
plt.title('Proportion of each Sentiment by Topic')

# Add a legend
plt.legend()

# Display the plot
plt.show()

histogram of sentiment by topic

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define colors for each topic
colors = ['blue', 'green', 'red']

# Loop through each topic (1, 2, 3)
for idx, topic in enumerate([1, 2, 3]):
    # Filter data for each topic
    compound_topic = general_log[general_log["topic_final"] == topic]["compounds_score"].dropna()
    
    # Compute mean sentiment score
    mean_score = np.mean(compound_topic)
    
    # Plot histogram with a different color for each topic
    plt.hist(compound_topic, bins=20, edgecolor='black', alpha=0.7, color=colors[idx])
    
    # Add mean as a vertical black dashed line
    plt.axvline(mean_score, color='black', linestyle='dashed', linewidth=2, label=f'Topic {topic} Mean: {mean_score:.2f}')
    
    # Labels and title
    plt.xlabel('VADER Sentiment Compound Score')
    plt.ylabel('Frequency')
    plt.title(f'Sentiment of Topic {topic} (n = {len(compound_topic)})')
    
    # Show legend
    plt.legend()

    # Show plot
    plt.show()


Statistics by Topic

In [ ]:
general_log

In [ ]:
def sentiment_stats_by_topic(data, topic_val):
    df = data[data["topic_final"] == topic_val]
    total = len(df)

    if total == 0:
        sentiment_result = {k: ("No data", 0, [None, None]) for k in ['negative', 'neutral', 'positive']}
        return 0, sentiment_result, ("No data", [None, None])

    # Sentiment distribution
    sentiment_result = {}
    for group in ["negative", "neutral", "positive"]:
        count = df[df["post_group"] == group].shape[0]
        p = count / total
        se = np.sqrt(p * (1 - p) / total)
        ci = [p - 1.96 * se, p + 1.96 * se]
        sentiment_result[group] = (count, round(p * 100, 2), [round(ci[0] * 100, 2), round(ci[1] * 100, 2)])

    # Compound score mean and 95% CI
    compound_scores = df["compounds_score"].dropna()
    n = len(compound_scores)
    if n == 0:
        compound_result = ("No data", [None, None])
    else:
        mean = compound_scores.mean()
        se = compound_scores.std(ddof=1) / np.sqrt(n)
        ci = [mean - 1.96 * se, mean + 1.96 * se]
        compound_result = (round(mean, 3), [round(ci[0], 3), round(ci[1], 3)])

    return total, sentiment_result, compound_result

# Loop through topics
for topic in sorted(general_log["topic_final"].dropna().unique()):
    total, sentiment_data, compound_data = sentiment_stats_by_topic(general_log, topic)
    
    print(f"Topic: {int(topic)} (Total posts: {total})\n")
    print("Sentiment Breakdown:")
    for sentiment, (count, prop, ci) in sentiment_data.items():
        if prop == "No data":
            print(f"  {sentiment.capitalize()}: No data")
        else:
            print(f"  {sentiment.capitalize()}: {count} posts {prop}% (95% CI: [{ci[0]}%, {ci[1]}%])")
    
    if compound_data[0] == "No data":
        print("\nMean Compound Score: No data")
    else:
        mean, ci = compound_data
        print(f"\nMean Compound Score: {mean}")
        print(f"  (95% CI, [{ci[0]}, {ci[1]}])")

    print("\n" + "-"*60 + "\n")

In [ ]:
import numpy as np

def overall_sentiment_and_compound(data):
    total = len(data)

    # Sentiment stats
    sentiment_result = {}
    for group in ["negative", "neutral", "positive"]:
        count = data[data["post_group"] == group].shape[0]
        p = count / total
        se = np.sqrt(p * (1 - p) / total)
        ci = [p - 1.96 * se, p + 1.96 * se]
        sentiment_result[group] = (count, round(p * 100, 2), [round(ci[0] * 100, 2), round(ci[1] * 100, 2)])

    # Compound score mean and 95% CI
    compound_scores = data["compounds_score"].dropna()
    n = len(compound_scores)
    mean = compound_scores.mean()
    se = compound_scores.std(ddof=1) / np.sqrt(n)
    ci = [mean - 1.96 * se, mean + 1.96 * se]
    compound_result = (round(mean, 3), [round(ci[0], 3), round(ci[1], 3)])

    return total, sentiment_result, compound_result

# Run it on your full DataFrame
total, sentiment_data, compound_data = overall_sentiment_and_compound(general_log)

# Print the results
print(f"Total posts: {total}\n")
print("Sentiment Breakdown:")
for sentiment, (count, percent, ci) in sentiment_data.items():
    print(f"  {sentiment.capitalize()}: {count} posts ({percent}%, 95% CI: [{ci[0]}%, {ci[1]}%])")

mean, mean_ci = compound_data
print(f"\nMean Compound Score: {mean}")
print(f"95% CI of Mean: [{mean_ci[0]}, {mean_ci[1]}]")

### statistics by keyword regardless of topic number

In [ ]:
import pandas as pd
import numpy as np

def proportion_with_ci(data, keyword):
    df = data[data["keyword"] == keyword]
    total = len(df)
    
    if total == 0:
        return {k: ("No data", 0, [None, None]) for k in ['negative', 'neutral', 'positive']}
    
    result = {}
    for group in ["negative", "neutral", "positive"]:
        count = df[df["post_group"] == group].shape[0]
        p = count / total
        se = np.sqrt(p * (1 - p) / total)
        ci = [p - 1.96 * se, p + 1.96 * se]
        # Convert to percentage
        result[group] = (count, round(p * 100, 2), [round(ci[0] * 100, 2), round(ci[1] * 100, 2)])
    
    return result

keywords = [
    "on! pouches", "rogue pouches", "zyn", "Lucy pouches", "unspecified",
    "fre pouches", "velo pouches", "niin pouches", "fr3sh pouches"
]

for kw in keywords:
    print(f"Keyword: {kw}")
    res = proportion_with_ci(general_log, kw)
    for sentiment, (count, prop, ci) in res.items():
        if prop == "No data":
            print(f"  {sentiment.capitalize()}: No data")
        else:
            print(f"  {sentiment.capitalize()}: {count} posts ({prop}%, 95% CI, [{ci[0]}%, {ci[1]}%])")
    print()

In [ ]:
import numpy as np

def overall_sentiment_summary(data):
    total = len(data)
    result = {}

    for group in ["negative", "neutral", "positive"]:
        count = data[data["post_group"] == group].shape[0]
        p = count / total
        se = np.sqrt(p * (1 - p) / total)
        ci = [p - 1.96 * se, p + 1.96 * se]
        result[group] = (count, round(p * 100, 2), [round(ci[0] * 100, 2), round(ci[1] * 100, 2)])
    
    return total, result

# Run it
total_posts, sentiment_breakdown = overall_sentiment_summary(general_log)

print(f"Total posts: {total_posts}\n")
for sentiment, (count, percentage, ci) in sentiment_breakdown.items():
    print(f"{sentiment.capitalize()}: {count} posts ({percentage}%, (95% CI, [{ci[0]}%, {ci[1]}%])")

In [ ]:
from scipy import stats
def mean_confidence_interval(data, confidence=0.95):
    data = np.array(data.dropna())
    n = len(data)
    mean = np.mean(data)
    stderr = stats.sem(data)  # standard error
    margin = stderr * stats.t.ppf((1 + confidence) / 2., n - 1)
    return mean, mean - margin, mean + margin
keywords = [
    "on! pouches", "rogue pouches", "zyn", "Lucy pouches",
    "unspecified", "fre pouches", "velo pouches", "niin pouches", "fr3sh pouches"
]

for keyword in keywords:
    scores = general_log[general_log["keyword"] == keyword]["compounds_score"]
    mean, lower, upper = mean_confidence_interval(scores)
    print(f"{keyword:<15} | Mean: {mean:.4f} | 95% CI: ({lower:.4f}, {upper:.4f})")

output

In [ ]:
output = general_log
output.head()

In [ ]:
output.drop("topics", axis = 1)

In [ ]:
output.to_csv('reddit_combined_final.csv', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('reddit_combined_final.csv')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'time_posted' to datetime
df['timestamp'] = pd.to_datetime(df['time_posted'])

# Set 'timestamp' as the index
df.set_index('timestamp', inplace=True)

# Resample by week (average sentiment per week)
weekly_sentiment = df['compounds_score'].resample('W').mean()

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(weekly_sentiment.index, weekly_sentiment, marker='o', linestyle='-')
plt.title('Weekly Average Sentiment Over Time')
plt.xlabel('Week')
plt.ylabel('Average Sentiment Score')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'time_posted' to datetime
df['timestamp'] = pd.to_datetime(df['time_posted'])

# Set 'timestamp' as index
df.set_index('timestamp', inplace=True)

# Resample by month and calculate average sentiment
monthly_sentiment = df['compounds_score'].resample('M').mean()

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(monthly_sentiment.index, monthly_sentiment, marker='o', linestyle='-')
plt.title('Monthly Average Sentiment Over Time')
plt.xlabel('Month')
plt.ylabel('Average Sentiment Score')
plt.grid


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert 'time_posted' to datetime
df['timestamp'] = pd.to_datetime(df['time_posted'])

# Set 'timestamp' as index
df.set_index('timestamp', inplace=True)

# Resample by year and calculate average sentiment
yearly_sentiment = df['compounds_score'].resample('Y').mean()

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(yearly_sentiment.index, yearly_sentiment, marker='o', linestyle='-')
plt.title('Yearly Average Sentiment Over Time')
plt.xlabel('Year')
plt.ylabel('Average Sentiment Score')
plt.grid(True)

# Optional: Format x-axis to show year only
import matplotlib.dates as mdates
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

plt.tight_layout()
plt.show()


### Graphs

In [ ]:
import pandas as pd
df = pd.read_csv('reddit_combined_final.csv')
df["keyword"] = df["keyword"].replace("zyn", "ZYN")
df["keyword"] = df["keyword"].replace("Lucy pouches", "Lucy")
df["keyword"] = df["keyword"].replace("fr3sh pouches", "FR3SH")
df["keyword"] = df["keyword"].replace("niin pouches", "NIIN")
df["keyword"] = df["keyword"].replace("on! pouches", "on!")
df["keyword"] = df["keyword"].replace("rogue pouches", "Rogue")
df["keyword"] = df["keyword"].replace("fre pouches", "FRE")
df["keyword"] = df["keyword"].replace("velo pouches", "VELO")
df["keyword"] = df["keyword"].replace("unspecified", "Others")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# ------------------------------------------------------------------
# 2.  Choose the keyword order you want to appear in the legend
#     (edit or reorder this list as you like)
# ------------------------------------------------------------------
keywords = [
    "VELO",
    'on!',
    'ZYN',
    'Others',
    'FRE',
    'Lucy',
    'FR3SH',
    'NIIN',
    'Rogue'
]

# ------------------------------------------------------------------
# 3.  Colour-blind-friendly palette (close to seaborn-colorblind)
# ------------------------------------------------------------------
palette = [
    '#0173b2',  # blue
    '#de8f05',  # orange
    '#029e73',  # green
    '#d55e00',  # red-orange
    '#cc78bc',  # purple
    '#ca9161',  # brown
    '#fbafe4',  # pink
    '#949494',  # gray
    '#ece133'   # yellow-green
]
colour_map = dict(zip(keywords, palette))

# ------------------------------------------------------------------
# 4.  Plot the filled KDEs
# ------------------------------------------------------------------
# (White background is the default Matplotlib style, so we *don't* call
#  plt.style.use('dark_background') here.)
plt.figure(figsize=(14, 7))

x_grid = np.linspace(-1, 1, 400)     # evaluation grid for all KDEs

for kw in keywords:
    scores = df.loc[df['keyword'] == kw, 'compounds_score'].dropna()
    if len(scores) > 1:               # need at least 2 points for a KDE
        kde = gaussian_kde(scores)
        y = kde.evaluate(x_grid)
        c = colour_map[kw]
        plt.fill_between(x_grid, y, alpha=0.35, color=c)   # semi-transparent fill
        plt.plot(x_grid, y, linewidth=1, color=c, 
                 label=f'{kw} (n={len(scores)})')

# ------------------------------------------------------------------
# 5.  Cosmetic touches
# ------------------------------------------------------------------
plt.xlabel('Compound Sentiment Score')
plt.ylabel('Density')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.legend(title='Keyword', bbox_to_anchor=(1.02, 1), loc='upper left',
           fontsize='small')
plt.tight_layout()
plt.show()


---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from math import ceil, sqrt

# ── 0.  Global font ───────────────────────────────────────────────
plt.rcParams["font.family"] = "Spectral"

# === Controls: shrink circle & enlarge text ======================
RADIUS_SCALE     = 0.75   # < 1.0 makes the radar circle smaller (no clipping)
LABEL_FONTSIZE   = 10     # keyword label size (was 8)
TITLE_FONTSIZE_AX= 13     # per-axes title size (was 11)
TITLE_FONTSIZE_FG= 16     # figure-level title size (was 14)
XTICK_PAD        = 14     # extra space between circle and labels
LINEWIDTH        = 2.2    # a touch thicker for visibility
FILL_ALPHA       = 0.25

# ── 1.  Load data ────────────────────────────────────────────────
for col in ("keyword", "compounds_score", "topic_final"):
    if col not in df.columns:
        raise RuntimeError(f"Missing column: {col}")

# ── 2.  Fixed spokes (overall top-8 keywords) ────────────────────
top8   = df["keyword"].value_counts().head(8).index.tolist()
angles = np.linspace(0, 2*np.pi, len(top8), endpoint=False).tolist() + [0]

# ── 3.  Colour palette ───────────────────────────────────────────
cmap = plt.get_cmap("tab10")

# ── 4.  Helper: radar of **counts** (labels include %) ────────────
def radar_topic(ax, topic_df, title, colour):
    counts = topic_df["keyword"].value_counts().reindex(top8, fill_value=0)
    props  = (counts / counts.sum()).fillna(0)
    mx     = counts.max() or 1

    # scale to 0–1 then shrink by RADIUS_SCALE so the circle is smaller
    base_scaled = (counts / mx).tolist() + [(counts / mx).iloc[0]]
    scaled = [v * RADIUS_SCALE for v in base_scaled]

    # plot
    ax.plot(angles, scaled, linewidth=LINEWIDTH, color=colour)
    ax.fill(angles, scaled, alpha=FILL_ALPHA, color=colour)

    # ticks & labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(
        [f"{kw}\n({counts[kw]} , {props[kw]:.1%})" for kw in top8],
        fontsize=LABEL_FONTSIZE, fontfamily="Spectral"
    )
    ax.tick_params(axis="x", pad=XTICK_PAD)  # push labels outward

    # radius limits match the scaled max to avoid clipping and keep ring small
    ax.set_ylim(0, RADIUS_SCALE)
    ax.set_yticks([])

    # styling
    ax.set_title(title, pad=16, fontsize=TITLE_FONTSIZE_AX, fontfamily="Spectral", fontweight="bold")
    ax.set_facecolor("white")
    ax.spines["polar"].set_color("#888")
    ax.grid(color="#bbb", linestyle="--", linewidth=0.5)

# ── 5.  Layout and per-topic colours ─────────────────────────────
topics = sorted(df["topic_final"].dropna().unique())

# Build colour map from tab10 …
colour_map = {tp: cmap(i % cmap.N) for i, tp in enumerate(topics)}
# … then override Topic 1 (if it exists) to be a bold red
if 1 in colour_map:
    colour_map[1] = "tab:red"
elif topics:
    colour_map[topics[0]] = "tab:red"

n_topics = len(topics)

if n_topics == 3:
    # slightly larger figure so big labels breathe
    fig = plt.figure(figsize=(11, 11), facecolor="white")
    gs  = matplotlib.gridspec.GridSpec(2, 2, height_ratios=[1, 1])

    ax1 = fig.add_subplot(gs[0, :], polar=True)
    ax2 = fig.add_subplot(gs[1, 0], polar=True)
    ax3 = fig.add_subplot(gs[1, 1], polar=True)

    radar_topic(ax1, df[df["topic_final"] == topics[0]], f"Topic {topics[0]}", colour_map[topics[0]])
    radar_topic(ax2, df[df["topic_final"] == topics[1]], f"Topic {topics[1]}", colour_map[topics[1]])
    radar_topic(ax3, df[df["topic_final"] == topics[2]], f"Topic {topics[2]}", colour_map[topics[2]])
else:
    n_cols = ceil(sqrt(n_topics))
    n_rows = ceil(n_topics / n_cols)
    # scale per-subplot size up a bit to fit larger labels
    fig, axes = plt.subplots(
        n_rows, n_cols,
        figsize=(n_cols*4.8, n_rows*4.8),
        subplot_kw=dict(polar=True),
        facecolor="white"
    )
    axes = np.atleast_1d(axes).flatten()

    for ax, tp in zip(axes, topics):
        radar_topic(ax, df[df["topic_final"] == tp], f"Topic {tp}", colour_map[tp])
    for ax in axes[n_topics:]:
        ax.set_visible(False)

# ── 6.  Figure-level title & layout tweaks ───────────────────────
fig.suptitle(
    "Keyword Post Counts & Proportions by Topic",
    y=0.98, fontsize=TITLE_FONTSIZE_FG, fontweight="bold",
    fontfamily="Spectral", color="black"
)

# leave a bit more room for the bigger labels
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ── 0.  Global font ───────────────────────────────────────────────
plt.rcParams["font.family"] = "Spectral"
plt.rcParams["font.size"] = 18

# ── 1.  Load data ────────────────────────────────────────────────
if "keyword" not in df.columns:
    raise RuntimeError("Missing 'keyword' column")

# ── 2.  Overall top-8 keywords (fixed spoke order) ───────────────
top8   = df["keyword"].value_counts().head(8).index.tolist()
angles = np.linspace(0, 2*np.pi, len(top8), endpoint=False).tolist() + [0]  # close loop

# ── 3.  Counts + proportions → 0–1 scaling for radius ────────────
counts = df["keyword"].value_counts().reindex(top8, fill_value=0)
props  = counts / counts.sum()
scaled = (counts / counts.max()).tolist() + [(counts / counts.max()).iloc[0]]

# ── 4.  Plot ─────────────────────────────────────────────────────
fig = plt.figure(figsize=(8, 8), facecolor="white")
ax  = plt.subplot(111, polar=True)

# Make the circle smaller (more space for text)
ax.set_position([0.15, 0.15, 0.7, 0.7])  # [left, bottom, width, height]
ax.set_rmax(0.8)                          # shrink radial extent

# Plot lines and fill
ax.plot(angles, scaled, linewidth=2.5, color="#1f77b4")
ax.fill(angles, scaled, alpha=0.25, color="#1f77b4")

# Labels: "keyword \n(count • pct)"
ax.set_xticks(angles[:-1])
ax.set_xticklabels(
    [f"{kw}\n({counts[kw]} , {props[kw]:.1%})" for kw in top8],
    fontsize=16,             # larger label font
    fontfamily="Spectral",
)

# Cleanup
ax.set_ylim(0, 1)
ax.set_yticks([])
ax.set_facecolor("white")
ax.spines["polar"].set_color("#888")
ax.grid(color="#bbbbbb", linestyle="--", linewidth=0.5)

# Title
ax.set_title(
    "Overall Keyword Post Counts & Proportions (linear-scaled radar)",
    pad=40,
    fontsize=14,
    fontfamily="Spectral",
    fontweight="bold"
)

plt.tight_layout()
plt.show()

see if the topics are different

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
from itertools import combinations
from statsmodels.stats.multitest import multipletests

df = pd.read_csv("reddit_combined_final.csv")
df = df.dropna(subset=["topic_final", "compounds_score"])
groups = df["topic_final"].unique()
results = []
for g1, g2 in combinations(groups, 2):
    scores1 = df[df["topic_final"] == g1]["compounds_score"]
    scores2 = df[df["topic_final"] == g2]["compounds_score"]
    
    # Welch's t-test (doesn't assume equal variances)
    t_stat, p_val = ttest_ind(scores1, scores2, equal_var=False)
    
    results.append((g1, g2, t_stat, p_val))

# Convert to DataFrame
results_df = pd.DataFrame(results, columns=["Group 1", "Group 2", "t-statistic", "p-value"])

# Optional: Bonferroni correction for multiple comparisons
results_df["p-adjusted (Bonferroni)"] = multipletests(results_df["p-value"], method="bonferroni")[1]

# Display results
print(results_df)
